In [1]:
import tabula as tb
import pandas as pd
import PyPDF2
from PyPDF2 import PdfReader
import re
import docx
from docx import Document
import zipfile
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [2]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': ' ', '\t': ' ', '    ': '', '&emsp': ' ', ' ,':',', '  ':' '
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = " ".join(str.split())
    str = str.strip('\n')
    str = str.strip(' ')
    return str

def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./result/2014.csv', index=False, encoding="utf-8")

In [8]:
### code entry
path = "./data/2014.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式
totaldata = handle(bs)
to_CSV(totaldata)

higher level topic name: TA1 Launch Propulsion Systems
topic name: TA1.3 Air Breathing Propulsion Systems
subtopic name: A3.03Low Emissions/Clean Power
higher level topic name: TA2 In-Space Propulsion Technologies
topic name: TA2.1 Chemical Propulsion
subtopic name: H2.02In-Space Chemical Propulsion
subtopic name: S4.03Spacecraft Technology for Sample Return Missions
topic name: TA2.2 Non-Chemical Propulsion
subtopic name: Z1.03Integrated Nuclear Power & Propulsion
subtopic name: H2.01High Power Electric Propulsion
subtopic name: H2.03Nuclear Thermal Propulsion (NTP)
subtopic name: S3.02Propulsion Systems for Robotic Science Missions
topic name: TA2.4 Supporting Technologies
subtopic name: H2.04Nuclear Thermal Propulsion (NTP) Ground Test Technologies
higher level topic name: TA3 Space Power and Energy Storage
topic name: TA3.1 Power Generation
subtopic name: Z1.01Advanced Photovoltaic Systems
subtopic name: H8.02Space Nuclear Power Systems
topic name: TA3.2 Energy Storage
subtopic nam

In [7]:
def handle(bs):
    totalData = []
    ### get overall topic
    regEvens = re.compile('md-taxonomy even.*')
    regOdds = re.compile('md-taxonomy odd.*')
    evens = bs.find_all("li", {"class": regEvens})
    odds = bs.find_all("li", {"class": regOdds})
    offset = 2
    for index in range(13):
        dic = {}
        higher_topic_block = evens[int(index / 2) + offset] if int(index % 2) == 0 else odds[int(index / 2) + offset]
        dic["Higher Topic"] = getHigherTopicName(higher_topic_block)
        regTopicEvens = re.compile('first-topic even.*')
        regTopicOdds = re.compile('first-topic odd.*')
        topic_even = higher_topic_block.find_all("li", {"class": regTopicEvens})
        topic_odd = higher_topic_block.find_all("li", {"class": regTopicOdds})
        for index2 in range(len(topic_even) + len(topic_odd)):
                topic_block = topic_even[int(index2 / 2)] if int(index2 % 2) == 0 else topic_odd[int(index2 / 2)]
                dic["Focus Area"] = getTopicName(topic_block)
                subtopic_even = topic_block.find_all("li", {"class": "second-topic even"})
                subtopic_odd = topic_block.find_all("li", {"class": "second-topic odd"})
                for index3 in range(len(subtopic_even) + len(subtopic_odd)):
                    subtopic_block = subtopic_even[int(index3 / 2)] if int(index3 % 2) == 0 else subtopic_odd[int(index3 / 2)]
                    dic["Subtopic"] = getSubtopicName(subtopic_block)
                    metaDic = getMetaData(subtopic_block)
                    for k in metaDic:
                        dic[k] = metaDic[k]
                    ### get other data
                    whole_info_block = subtopic_block.find("div", {"class": "whole-topic-content"})
                    new_dic = dic.copy()
                    new_dic["Content"] = filterHTMLstr(whole_info_block.get_text())
                    totalData.append(new_dic)
    return totalData

In [4]:
### get higher level topic
def getHigherTopicName(topic_block):
    higher_topic_name = filterHTMLstr(topic_block.find("h2").get_text())
    print("higher level topic name:", higher_topic_name)  # higher level topic name
    return higher_topic_name

### get topic name
def getTopicName(topic_block):
    topic_name = filterHTMLstr(topic_block.find("h4", {"class": "topicTitle"}).get_text())
    print("topic name:", topic_name)  # subtopic name
    return topic_name

### handle every sub-topic
def getSubtopicName(subtopic_block):
    subtopic_name = filterHTMLstr(subtopic_block.find("p", {"class": "subtopic"}).get_text())
    print("subtopic name:", subtopic_name)  # subtopic name
    return subtopic_name

In [5]:
### get Lead Center, Participating Center(s), Solicitation Year
def getMetaData(subtopic_block):
    dic = {}
    info_group = subtopic_block.find_all("p", {"class": "l-center"})
    for raw_info in info_group:
        info = filterHTMLstr(raw_info.get_text())
        pair = info.split(":")
        dic[pair[0]] = pair[1]
        # print(pair[0], pair[1])
    # print(dic)
    return dic

In [12]:
for i in evens:
    getHigherTopicName(i)

higher level topic name: Aeronautics Research
higher level topic name: Science
higher level topic name: TA1 Launch Propulsion Systems
higher level topic name: TA3 Space Power and Energy Storage
higher level topic name: TA5 Communication and Navigation
higher level topic name: TA7 Human Exploration Destination Systems
higher level topic name: TA9 Entry, Descent and Landing Systems
higher level topic name: TA10.2 Energy Generation and Storage
higher level topic name: TA12 Materials, Structures, Mechanical Systems and Manufacturing
higher level topic name: TA14 Thermal Management Systems


In [13]:
for i in odds:
    getHigherTopicName(i)

higher level topic name: Human Exploration and Operations
higher level topic name: Space Technology
higher level topic name: TA2 In-Space Propulsion Technologies
higher level topic name: TA4 Robotics, Telerobotics and Autonomous Systems
higher level topic name: TA6 Human Health, Life Support and Habitation Systems
higher level topic name: TA8 Science Instruments, Observatories & Sensor Systems
higher level topic name: TA10 Nanotechnology
higher level topic name: TA11 Modeling, Simulation, Information Technology and Processing
higher level topic name: TA13 Ground and Launch Systems Processing
higher level topic name: TA15 Aeronautics


In [14]:
regEvens = re.compile('md-taxonomy even.*')
regOdds = re.compile('md-taxonomy odd.*')
evens = bs.find_all("li", {"class": regEvens})
odds = bs.find_all("li", {"class": regOdds})
offset = 2
for index in range(16):
    higher_topic_block = evens[int(index / 2) + offset] if int(index % 2) == 0 else odds[int(index / 2) + offset]
    getHigherTopicName(higher_topic_block)
    regTopicEvens = re.compile('first-topic even.*')
    regTopicOdds = re.compile('first-topic odd.*')
    topic_even = higher_topic_block.find_all("li", {"class": regTopicEvens})
    topic_odd = higher_topic_block.find_all("li", {"class": regTopicOdds})
    for index2 in range(len(topic_even) + len(topic_odd)):
            topic_block = topic_even[int(index2 / 2)] if int(index2 % 2) == 0 else topic_odd[int(index2 / 2)]
            getTopicName(topic_block)
            subtopic_even = topic_block.find_all("li", {"class": "second-topic even"})
            subtopic_odd = topic_block.find_all("li", {"class": "second-topic odd"})
            for index3 in range(len(subtopic_even) + len(subtopic_odd)):
                subtopic_block = subtopic_even[int(index3 / 2)] if int(index3 % 2) == 0 else subtopic_odd[int(index3 / 2)]
                getSubtopicName(subtopic_block)

higher level topic name: TA1 Launch Propulsion Systems
topic name: TA1.2 Liquid Rocket Propulsion Systems
subtopic name: H10.01Cryogenic Purge Gas Recovery and Reclamation
higher level topic name: TA2 In-Space Propulsion Technologies
topic name: TA2.1 Chemical Propulsion
subtopic name: H2.01In-Space Chemical Propulsion
subtopic name: S4.03Spacecraft Technology for Sample Return Missions
topic name: TA2.2 Non-Chemical Propulsion
subtopic name: H2.02Nuclear Thermal Propulsion (NTP)
subtopic name: H2.03High Power Electric Propulsion
subtopic name: S3.02Propulsion Systems for Robotic Science Missions
topic name: TA2.4 Supporting Technologies
subtopic name: H2.04Cryogenic Fluid Management for In-Space Transportation
higher level topic name: TA3 Space Power and Energy Storage
topic name: TA3.1 Power Generation
subtopic name: H8.01Space Nuclear Power Systems
subtopic name: H8.02Solid Oxide Fuel Cells and Electrolyzers
subtopic name: H8.03Advanced Photovoltaic Systems
subtopic name: S3.01Power